In [61]:
# create folders
mkdir demos
mkdir demos/pod

In [2]:
kubectl get pods

No resources found in default namespace.


In [7]:
# check DOCS for the run image command
kubectl run -h | grep image

Create and run a particular image in a pod.
  kubectl run nginx --image=nginx
  kubectl run hazelcast --image=hazelcast/hazelcast --port=5701
  kubectl run hazelcast --image=hazelcast/hazelcast --env="DNS_DOMAIN=cluster" --env="POD_NAMESPACE=default"
  kubectl run hazelcast --image=hazelcast/hazelcast --labels="app=hazelcast,env=prod"
  kubectl run nginx --image=nginx --dry-run=client
  kubectl run nginx --image=nginx --overrides='{ "apiVersion": "v1", "spec": { ... } }'
  kubectl run -i -t busybox --image=busybox --restart=Never
  kubectl run nginx --image=nginx -- <arg1> <arg2> ... <argN>
  kubectl run nginx --image=nginx --command -- <cmd> <arg1> ... <argN>
    --image='':
	The image for the container to run.
    --image-pull-policy='':
	The image pull policy for the container.  If left empty, this value will not be specified by the client and defaulted by the server.
  kubectl run NAME --image=image [--env="key=value"] [--port=port] [--dry-run=server|client] [--overrides=inline-jso

In [8]:
# create a pod using nginx as image
kubectl run nginx --image nginx

pod/nginx created


In [9]:
# check the deployment
kubectl get pods -o wide 

NAME    READY   STATUS    RESTARTS   AGE   IP            NODE       NOMINATED NODE   READINESS GATES
nginx   1/1     Running   0          83s   10.244.0.87   minikube   <none>           <none>


In [15]:
# check some info about the pod
kubectl describe pod nginx | head 

Name:             nginx
Namespace:        default
Priority:         0
Service Account:  default
Node:             minikube/192.168.49.2
Start Time:       Fri, 03 Feb 2023 00:09:16 +0100
Labels:           run=nginx
Annotations:      <none>
Status:           Running
IP:               10.244.0.87


In [18]:
# check the pod image used
kubectl describe pod nginx | grep Image:

    Image:          nginx


In [29]:
# check the node in which the pod is running
kubectl get pods -o wide | awk '{print $7}'

NODE
minikube


In [30]:
# clean up
kubectl delete pod nginx

pod "nginx" deleted


In [49]:
# create new pod with a wrong image, using a file output
kubectl run redis --image redis123 --dry-run=client -o yaml | tee ./demos/pod/redis.yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: null
  labels:
    run: redis
  name: redis
spec:
  containers:
  - image: redis123
    name: redis
    resources: {}
  dnsPolicy: ClusterFirst
  restartPolicy: Always
status: {}


In [41]:
# deploy the file
kubectl create -f ./demos/pod/redis.yaml

pod/redis created


In [42]:
# check the pods
kubectl get pods -o wide

NAME    READY   STATUS             RESTARTS   AGE   IP            NODE       NOMINATED NODE   READINESS GATES
redis   0/1     ImagePullBackOff   0          48s   10.244.0.88   minikube   <none>           <none>


In [43]:
# check the error
kubectl describe pod redis | grep error

  Warning  Failed     7s (x4 over 107s)   kubelet            Failed to pull image "redis123": rpc error: code = Unknown desc = Error response from daemon: pull access denied for redis123, repository does not exist or may require 'docker login': denied: requested access to the resource is denied


In [50]:
# update the image with the right image in the file
sed -i 's/redis123/redis/' ./demos/pod/redis.yaml 

In [53]:
# apply the file changes
kubectl apply -f ./demos/pod/redis.yaml 

pod/redis configured


In [59]:
# validate the pods
kubectl get pods -o wide | awk '{print $1, $3}'

NAME STATUS
redis Running


In [60]:
# clean up
kubectl delete pod redis

pod "redis" deleted
